In [31]:
import pandas as pd

import LotData.LotData as ld
import LotData.ExtractorsAndTables as ent
import utility.LoggingUtility as lut
import utility.webscrapingUtil as wut
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import CW_Scraper
import sqlalchemy
import time
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import text
from Runnables.RunningSettings import wanted_categories
import database.DatabaseManager as dbm

In [64]:
session,engine = dbm.getSessionEngine()

In [3]:
err_lids = []
for (cat_int,cat_name) in wanted_categories.items():
    items = 0
    wttches_magov = CW_Scraper.MagazineOverview(cat_int)
    wttches_magov.set_active_nr_pages()
    for i,page_i in enumerate(wttches_magov):
        times = []
        for LID in page_i:
            t1 = time.time()
            meta_data = ent.MetadataExtractor(LID,lut.getTimeStamp(),cat_int,cat_name)

            l_data = ld.LotData(meta_data)
            for record_key in l_data.keys():
                table = wut.recordIntoTabe(record_key)
                query = f"SELECT EXISTS (SELECT 1 FROM {table} WHERE lid = :lid)"

                result = session.execute(text(query), {'lid': LID})
                exists = result.scalar()  # or fetch the result as required
                if(not exists):
                    try:

                        df = l_data[record_key]
                        if(record_key == "meta_record"):
                            df["status"] = "idle"
                        df.to_sql(record_key.replace("_record",""), con=engine, if_exists='append',index=False)
                    except Exception as e:
                        err_lids += [(LID,e)]

            times += [time.time() - t1]
            time.sleep(np.random.uniform(0,0.01))
        if(np.mean(times)>2):
            print(f"Page number: {i} out of {len(wttches_magov)} of category {cat_name} \t Average time lot: {np.mean(times)} with variance {np.var(times)}")

    print(f"Finished category: {cat_name}")


non_expected_errors = [err for err in err_lids if len(err) > 1 and (type(err[1]) != sqlalchemy.exc.IntegrityError) and (type(err[1]) != KeyError)]

Page number: 0 out of 225 of category watches 	 Average time lot: 2.233286360899607 with variance 8.252248466220422
Page number: 2 out of 225 of category watches 	 Average time lot: 2.3577302992343903 with variance 8.088782810759083
Page number: 9 out of 225 of category watches 	 Average time lot: 2.4860459864139557 with variance 7.876158273786959
Page number: 10 out of 225 of category watches 	 Average time lot: 2.24306058883667 with variance 7.279503547081181


C:\Users\DripTooHard\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\DripTooHard\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Page number: 12 out of 225 of category watches 	 Average time lot: 3.5294811725616455 with variance 7.360325324472754
Page number: 13 out of 225 of category watches 	 Average time lot: 3.760557174682617 with variance 6.974643104156708
Page number: 14 out of 225 of category watches 	 Average time lot: 4.931382497151692 with variance 20.472570155775507
Page number: 16 out of 225 of category watches 	 Average time lot: 2.525701344013214 with variance 8.956291697288647
Page number: 17 out of 225 of category watches 	 Average time lot: 2.270370384057363 with variance 8.070518208290062
Page number: 18 out of 225 of category watches 	 Average time lot: 3.4459947446982064 with variance 7.787442922633829
Page number: 19 out of 225 of category watches 	 Average time lot: 2.04956046740214 with variance 6.981890830986789
Page number: 24 out of 225 of category watches 	 Average time lot: 2.05164768298467 with variance 7.470711329969441
Page number: 25 out of 225 of category watches 	 Average time l

In [74]:
print(wttches_magov.ac)

In [72]:
result = session.execute(text("select lid from spec where category_name = 'watches'")).fetchall()
result = [lid[0] for lid in result]

In [73]:
err_df = pd.DataFrame.from_records(err_lids)
unique_lids = np.array(err_df[0].unique())

not_in = []
for idx,lid in enumerate(unique_lids):
    if(int(lid) not in result):
        not_in += [(idx,lid)]

print(not_in)

[(527, '85468553')]


In [77]:
#meta_timestamp between '2024-07-01 00:00:00' and '2024-08-01 00:00:00' and category_name = 'watches'

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
